<a href="https://colab.research.google.com/github/Paolino1994/NLP-CEIA-Fiuba/blob/main/TP3/TP3_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

#Descargar Dataset

In [3]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip   
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [4]:
# Posibles bandas
os.listdir("./songs_dataset/")

['bieber.txt',
 'nirvana.txt',
 'alicia-keys.txt',
 'dickinson.txt',
 'dolly-parton.txt',
 'Kanye_West.txt',
 'nursery_rhymes.txt',
 'disney.txt',
 'drake.txt',
 'nickelback.txt',
 'bob-dylan.txt',
 'ludacris.txt',
 'al-green.txt',
 'radiohead.txt',
 'paul-simon.txt',
 'dr-seuss.txt',
 'beatles.txt',
 'notorious_big.txt',
 'amy-winehouse.txt',
 'nicki-minaj.txt',
 'adele.txt',
 'kanye-west.txt',
 'jimi-hendrix.txt',
 'cake.txt',
 'joni-mitchell.txt',
 'lady-gaga.txt',
 'bob-marley.txt',
 'missy-elliott.txt',
 'lorde.txt',
 'r-kelly.txt',
 'bruno-mars.txt',
 'Lil_Wayne.txt',
 'lin-manuel-miranda.txt',
 'eminem.txt',
 'patti-smith.txt',
 'dj-khaled.txt',
 'prince.txt',
 'blink-182.txt',
 'bjork.txt',
 'leonard-cohen.txt',
 'johnny-cash.txt',
 'janisjoplin.txt',
 'notorious-big.txt',
 'britney-spears.txt',
 'lil-wayne.txt',
 'michael-jackson.txt',
 'kanye.txt',
 'bruce-springsteen.txt',
 'rihanna.txt']

In [5]:
df = pd.read_csv('songs_dataset/bruce-springsteen.txt', sep='/n', header=None)
df.head()

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,0
0,[Verse 1]
1,They come from everywhere
2,A longing to be free
3,They come to join us here
4,From sea to shining sea And they all have a dream


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2413


# Preprocesamiento

In [7]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

# Word2Vec

In [8]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [9]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=300,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [10]:
w2v_model.build_vocab(sentence_tokens)

In [11]:
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de docs en el corpus: 2413
Cantidad de words distintas en el corpus: 609


# Entrenamiento

In [15]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=100,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 64151.66015625
Loss after epoch 1: 65212.01171875
Loss after epoch 2: 64538.640625
Loss after epoch 3: 65855.5625
Loss after epoch 4: 64760.75
Loss after epoch 5: 64911.9375
Loss after epoch 6: 64063.5625
Loss after epoch 7: 63967.6875
Loss after epoch 8: 63695.375
Loss after epoch 9: 62394.75
Loss after epoch 10: 63107.75
Loss after epoch 11: 62402.0
Loss after epoch 12: 62945.4375
Loss after epoch 13: 63050.0
Loss after epoch 14: 62248.5625
Loss after epoch 15: 63350.5
Loss after epoch 16: 60997.6875
Loss after epoch 17: 60483.125
Loss after epoch 18: 61541.375
Loss after epoch 19: 61643.0
Loss after epoch 20: 59793.0
Loss after epoch 21: 61276.0
Loss after epoch 22: 60776.125
Loss after epoch 23: 60377.125
Loss after epoch 24: 59887.875
Loss after epoch 25: 60400.25
Loss after epoch 26: 60680.875
Loss after epoch 27: 59848.25
Loss after epoch 28: 60004.25
Loss after epoch 29: 61147.5
Loss after epoch 30: 60673.75
Loss after epoch 31: 60099.25
Loss after epoch 32:

(1227949, 2156600)

In [18]:
w2v_model.wv.most_similar(positive=["cry"], topn=10)

[('mourn', 0.5651511549949646),
 ('weep', 0.5596333742141724),
 ('brothers', 0.5214001536369324),
 ('because', 0.5062947273254395),
 ('o', 0.5029638409614563),
 ('mary', 0.48766839504241943),
 ('tired', 0.42927539348602295),
 ('sha', 0.42677968740463257),
 ('trapped', 0.4249432682991028),
 ('tenth', 0.42290160059928894)]

Podemos ver que asocia el llorar con el duelo, con weep, que tambien viene de llorar, pero lo extraño es que despues se va para el lado de brothers, lo que resulta un poco extraño, ya que bruce springsteen no tiene hermanos.

In [33]:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('hate', 0.622441291809082),
 ('conquer', 0.5479432344436646),
 ('save', 0.4436076879501343),
 ('sha', 0.43522584438323975),
 ('lift', 0.4169168472290039),
 ('bet', 0.40212035179138184),
 ('la', 0.3938601016998291),
 ('anything', 0.384234756231308),
 ('ready', 0.37896519899368286),
 ('sherry', 0.3755594491958618)]

In [28]:
w2v_model.wv.most_similar(negative=["love"])

[('stood', 0.024111412465572357),
 ('miles', 0.018508948385715485),
 ('walked', -0.00810830295085907),
 ('left', -0.009866224601864815),
 ('fifteen', -0.012942483648657799),
 ('erie', -0.02010757103562355),
 ('are', -0.02032061666250229),
 ('takes', -0.020703434944152832),
 ('franky', -0.020762380212545395),
 ('canal', -0.030424151569604874)]

Curioso que la palabra mas relacionada a amor sea odio, pero bueno, son los 2 sentimientos mas fuertes del ser humano y comunmente se pasa de uno a otro de forma muy rapida. Depsues tambien habla de conquistar y de salvar el amor. Tambien podemos destacar la particula "la" que se suele usar para cantar. ASi que para bruce, esa particula esta mas asociada a la positividad que a lo negativo.
EN el caso de lo negativo, la verdad no noto relaciones notables.

In [34]:
w2v_model.wv.most_similar(positive=["night"], topn=10)

[('prove', 0.4583151936531067),
 ('missing', 0.4291517734527588),
 ('dance', 0.4126948118209839),
 ('saturday', 0.40749508142471313),
 ('last', 0.38029491901397705),
 ('dreamed', 0.3797728419303894),
 ('middle', 0.3795884847640991),
 ('played', 0.36400777101516724),
 ('saw', 0.355297714471817),
 ('fact', 0.34745365381240845)]

In [35]:
w2v_model.wv.most_similar(negative=["night"], topn=10)

[('more', 0.01975090801715851),
 ('friend', 0.002518860623240471),
 ('far', 0.002028927206993103),
 ('wants', -0.0026906831189990044),
 ('better', -0.009141745045781136),
 ('dreams', -0.015119064599275589),
 ("can't", -0.016778932884335518),
 ('franky', -0.01779075153172016),
 ('tell', -0.026394188404083252),
 ('front', -0.02657454088330269)]

Con respecto a noche, notamos palabras como dance(bailar), saturday, last y night. Creo que para particularmente esta palabra funciona muy bien. Por lo menos un poco particular lo vemos del otro lado, donde relaciona de manera negativa la noche con amigos, lo cual para algunos es asi, para otros no tanto.

In [32]:
w2v_model.wv.most_similar(negative=["hate"])

[('stood', 0.03978778421878815),
 ('girls', -0.05945466086268425),
 ('walked', -0.06240247189998627),
 ('took', -0.06699390709400177),
 ('miles', -0.0818716511130333),
 ('last', -0.08213238418102264),
 ('sea', -0.08377984166145325),
 ('steps', -0.08867721259593964),
 ('comes', -0.09906250983476639),
 ('town', -0.10737701505422592)]

# Visualizacion

In [29]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [30]:
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


En conclusion, el modelo performa segun lo esperado, es relativamente una base chica pero las canciones de bruce suelen tener algun tipo de sentido y no solo riman. Por lo tanto las relaciones entre las palabras son interesantes. Capaz seria interesante agregar informacion de canciones de artistas del mismo genero y de la misma epoca que cuando el empezo, como puede ser Bob Seger.